In [1]:
import yfinance as yf
import pandas as pd
import os

In [ ]:
class StockTicker:
    def __init__(self, stockName:str, timePeriod:str, timeInterval:str="1h") -> None:
        self.name=stockName
        periods=['1d', '5d', '1mo', '3mo', '6mo', '1y', '2y', '5y', '10y', 'ytd', 'max']
        if(timePeriod not in periods):
            raise ValueError(f"Supported periods are [{periods}]")
        self.period=timePeriod

        intervals=["1m", "2m", "5m", "15m", "30m", "60m", "90m", "1h", "1d", "5d", "1wk", "1mo", "3mo"]

        #I guess api doesn't care the given interval if the period is too large
        if(timeInterval not in intervals):
            raise ValueError(f"Supported intervals are [{intervals}]")
        self.interval=timeInterval

        self.ticker=yf.Ticker(self.name)

    def getOHLC(self):
        try:
            self.data=self.ticker.history(period=self.period).reset_index().drop(columns=["Dividends","Stock Splits"])
        except:
            if(self.period=="max"):
                return pd.DataFrame()
            self.period="max"

            return self.getOHLC()

        # Select columns with 'float64' dtype  
        float64_cols = list(self.data.select_dtypes(include='float64'))
        
        self.data[float64_cols] = self.data[float64_cols].astype('float32')
        # Select columns with 'float64' dtype  
        int64_cols = list(self.data.select_dtypes(include='int64'))
        self.data[int64_cols] = self.data[int64_cols].astype('int32')

        self.data['Volume']=self.data['Volume'].rolling(window=5, min_periods=1, center=True).mean()
        #TODO balance_sheet update: Do we really need these for the first stage
        return self.data
    
